In [ ]:
import pandas as pd
import pysolr

In [ ]:
# url = "http://b2find.eudat.eu/solr/select"
url = "http://localhost:8983/solr/ckan"

solr = pysolr.Solr(url)

In [ ]:
b2f_facet_map = dict(
    communities="groups",
    keywords="tags",
    creator="author",
    instrument="extras_Instrument",
    discipline="extras_Discipline",
    language="extras_Language",
    publisher="extras_Publisher",
    contributor="extras_Contributor",
    resourcetype="extras_ResourceType",
    openaccess="extras_OpenAccess",
)

In [ ]:
facet = b2f_facet_map.get("creator")
facet_sort = "count"
facet_sort_ascending = False
facet_limit = 10
facet_filter = "ursul"

solr_params = {
        'echoParams': 'none',
        'rows': 0,
        'wt': 'json',
        'q': '*',
        # 'fq': '',
        'facet': 'true',
        'facet.contains': facet_filter,
        'facet.contains.ignoreCase': 'true',
        'facet.sort': facet_sort,
        'facet.limit': facet_limit,
        'facet.mincount': 1,
        'facet.field': [
            f'{facet}',
        ],
    }

In [ ]:
search_results = solr.search(**solr_params)


In [ ]:
values = search_results.facets['facet_fields'][facet]
labels = values[0::2]
counts = values[1::2]

In [ ]:
df = pd.DataFrame(list(zip(labels, counts)), columns=['label', 'count'])
df

In [ ]:
df['label_downcase'] = df.label.str.lower()
df

In [ ]:
if facet_sort == "count":
    df.sort_values(by=['count', 'label_downcase'], inplace=True, ascending=facet_sort_ascending)
else:
    df.sort_values(by=['label_downcase'], inplace=True, ascending=True)
df

In [ ]:
df['label_truncated'] = df.label.str.slice(0, 22)

In [ ]:
out = df[['label', 'count', 'label_truncated']].iloc[:facet_limit].to_json(orient="records", indent=4)
print(out)